In [84]:
import re
import pandas as pd
from datetime import datetime

df = pd.read_excel('caffe.xlsx')

# Adjusting quantity

In [85]:
def adjust_quantity(desc, qty, discount):
    pattern = r"(\d+)\+(\d+)"
    match = re.search(pattern, desc)
    if match:
        x, y = map(int, match.groups())
        qty = qty * x + y
        desc = re.sub(pattern, "", desc).strip()
        discount = match.group(0)
    return desc, qty, discount

In [86]:
df["Discount Type"] = ""

df[["Item Desc", "Quantity", "Discount Type"]] = df.apply(
    lambda row: pd.Series(adjust_quantity(row["Item Desc"], row["Quantity"], row["Discount Type"])),
    axis=1
)

# Add weekdays

In [87]:
def add_day_of_week(date):
    wd = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return wd[date.weekday()]

In [88]:
df["Week day"] = df["Date"].apply(add_day_of_week)

# Add weeknumbers

In [89]:
df["Week number"] = df["Date"].dt.isocalendar().week
df.to_csv('caffe_change.csv', index=False)

# Split btl and glss

In [90]:
df['GLS/BTL'] = df['Item Desc'].apply(
    lambda x: 'GLS' if any(w in x.upper() for w in ['GLS', 'GLASS']) 
    else ('BTL' if any(w in x.upper() for w in ['BTL', 'BOTTLE']) else '')
)

# Add TOPPINGS categorie

In [ ]:
def change_to_toppings(desc, category):
    pattern = r'\b(ADD?)\b'
    match = re.search(pattern, desc)
    if match:
        desc = re.sub(pattern, "", desc).strip()
        category = "ADD-ONS"
    return desc, category


df[["Item Desc", "Category"]] = df.apply(
    lambda row: pd.Series(change_to_toppings(row["Item Desc"], row["Category"])),
    axis=1
)


# Write to new csv

In [92]:
df.to_csv('caffe_change.csv', index=False)